In [9]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from operator import itemgetter

In [10]:
# Collecting dance files and setting test constants
data_path = "/Users/emmawaters/Desktop/Dance/greenTest/"
vid_names = []
dir_list = os.listdir(data_path)
for i in dir_list:
    if i.endswith(".mp4"):
        vid_names.append(i)

NUM_FRAMES = float('inf')
NUM_PARTS = 15
NUM_TIME_CHUNKS = 5
NUM_VIDS = len(vid_names)
print(vid_names)
print(NUM_VIDS)

['7.mp4', '6.mp4', '4.mp4', '5.mp4', '1.mp4', '2.mp4', '3.mp4', '16.mp4', '14.mp4', '15.mp4', '11.mp4', '10.mp4', '12.mp4', '13.mp4', '8.mp4', '9.mp4']
16


In [20]:
def read_video_to_movement(vidpath):
    data = []
    net = cv.dnn.readNetFromTensorflow("graph_opt.pb")
    cap = cv.VideoCapture(vidpath)

    BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                   "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                   "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Background": 14 }

    POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
                   ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
                   ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
                   ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"] ]

    error = 0
    while cv.waitKey(1) < 0:
        hasFrame, frame = cap.read()
        if not hasFrame:
            break

        #frame = cv.rotate(frame, cv.ROTATE_180) #ONLY needed if raw iPhone data
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        net.setInput(cv.dnn.blobFromImage(frame, 1.0, (368, 368), (127.5, 127.5, 127.5), swapRB=True, crop=False))
        out = net.forward()
        out = out[:, :15, :, :]

        assert(len(BODY_PARTS) == out.shape[1])

        points = []
        for i in range(len(BODY_PARTS)):
            heatMap = out[0, i, :, :]
            _, conf, _, point = cv.minMaxLoc(heatMap)
            x = (frameWidth * point[0]) / out.shape[3]
            y = (frameHeight * point[1]) / out.shape[2]
            if conf > 0.1:
                points.append(np.array([x, y]))
            else:
                points.append(np.array([None,None]))
                error += 1

        data.append(points)
        
        for pair in POSE_PAIRS:
            partFrom = pair[0]
            partTo = pair[1]
            assert(partFrom in BODY_PARTS)
            assert(partTo in BODY_PARTS)

            idFrom = BODY_PARTS[partFrom]
            idTo = BODY_PARTS[partTo]

            if points[idFrom].any() and points[idTo].any():
                cv.line(frame, [int(points[idFrom][0]), int(points[idFrom][1])], [int(points[idTo][0]), int(points[idTo][1])], (0, 255, 0), 3)
                cv.ellipse(frame, [int(points[idFrom][0]), int(points[idFrom][1])], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
                cv.ellipse(frame, [int(points[idTo][0]), int(points[idTo][1])], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

        t, _ = net.getPerfProfile()
        freq = cv.getTickFrequency() / 1000
        cv.putText(frame, '%.2fms' % (t / freq), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

        cv.imshow('OpenPose using OpenCV', frame)

    cv.destroyWindow('OpenPose using OpenCV')
        
    return (data, error)

In [21]:
# Collect movements in a list, calculate number of missed joints
movement_list = []
total_missed = 0
total_frames = 0

for i in range(len(vid_names)):
    print(i)
    (movement, missed) = read_video_to_movement(data_path + vid_names[i])
    movement_list.append(movement)
    vid_frames = len(movement)
    total_missed += missed
    total_frames += vid_frames
    if vid_frames < NUM_FRAMES:
        NUM_FRAMES = vid_frames
        
# Compare number of missed joints to the number of expected joints
expected_num_joints = vid_frames * 15 * NUM_VIDS
error_factor = total_missed/expected_num_joints
print(error_factor)

0
1
2
3
4
5
6
7


KeyboardInterrupt: 